# Counter‑factual Hallmark Pathway Perturbations in Cancer scRNA‑seq

**Objectives**

1. **Generate counter‑factual single‑cell states** by “dialling” chosen Hallmark pathways sharply *up* or *down* inside selected cancer populations without producing biologically impossible expression values.  
2. **Embed perturbed and baseline cells** with single‑cell foundation models (scGPT, Geneformer, etc.).  
3. **Quantify & visualise the displacement** of each perturbed population in embedding space relative to multiple baselines (proliferative vs quiescent, treated vs untreated, …).  
4. **Rank perturbations** by a rich panel of metrics (axial progress, Wasserstein shift, silhouette improvement, Calinski–Harabasz, Davies–Bouldin, K‑NN overlap, etc.) to discover which pathway combinations most cleanly separate the populations.  

**Key Assumptions**

* Baseline AnnData contains ≥ two reference conditions (e.g. *Proliferative* and *Growth‑arrested*) that act as anchors for axial scoring.  
* Directionality of Hallmark sets is derived from known biology or differential analysis (positive = expected to **increase** under pathway activation).  
* Knock‑up/‑down magnitudes are expressed in **MAD units** or extreme quantiles, never outside observed global minima/maxima.  
* Foundation‑model embeddings already exist (pre‑computed) or can be generated on the fly with an encoder callable.  

---



In [32]:
import pandas as pd
import numpy as np
import scanpy as sc

# ─── Standard library ──────────────────────────────────────────────────────────
import os
import sys
from pathlib import Path
import time
from datetime import datetime
import csv
import pickle
import os, random, numpy as np, scanpy as sc, pandas as pd, scipy.sparse as sp

# ─── Common analysis packages ──────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import anndata as ad
import mlflow
from hydra import initialize, compose
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ─── Helical Bioagents foundation models & configs ────────────────────────────────────
from bioagents.probing.probing_agent import ProbingAgent
from bioagents.probing.probing_config import ProbingPredictConfig, ProbingTrainConfig
from bioagents.probing.evaluate import evaluate
from bioagents.inference.run import run_inference
from bioagents.inference.inference_config import InferenceConfig

# ─── Project‐specific utilities ───────────────────────────────────────────────
# ensure ../utils is on PYTHONPATH before importing local modules
sys.path.append(str(Path("./utils")))
from perturbation_utils import *

In [ ]:
 # Hallmark directional template ------------------------------------
    "hallmark_up"   : [                               # ↑ to simulate response
        "HALLMARK_P53_PATHWAY",
        "HALLMARK_APOPTOSIS",
        "HALLMARK_UNFOLDED_PROTEIN_RESPONSE",
        "HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY",
        "HALLMARK_INTERFERON_GAMMA_RESPONSE",
    ],
    "hallmark_down" : [                               # ↓ to suppress growth
        "HALLMARK_G2M_CHECKPOINT",
        "HALLMARK_E2F_TARGETS",
        "HALLMARK_MYC_TARGETS_V1",
        "HALLMARK_MTORC1_SIGNALING",
    ],

In [3]:
# read LUCA_lung_cancer_data
adata = sc.read('/home/helical_team/issac/data/raw_data_collection/36368318_LuCA_lung_cancer_atlas/80b57568-2621-4911-b4b1-4f2cf5087962.h5ad',backed = 'r')

In [5]:
adata.obs

,sample,uicc_stage,ever_smoker,age,donor_id,origin,dataset,ann_fine,cell_type_predicted,doublet_status,...,EGFR_mutation,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid
001C_AAACCTGCATCGGGTC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Monocyte non-classical,Monocyte,singlet,...,NaN,non-classical monocyte,10x 3' v2,normal,Homo sapiens,male,lung,unknown,22-year-old stage,re{$hj#%bw
001C_AAACCTGTCAACACCA-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,NaN,alveolar macrophage,10x 3' v2,normal,Homo sapiens,male,lung,unknown,22-year-old stage,mr6dH!~~)J
001C_AAACGGGAGACTAAGT-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Endothelial cell lymphatic,Endothelial cell,singlet,...,NaN,endothelial cell of lymphatic vessel,10x 3' v2,normal,Homo sapiens,male,lung,unknown,22-year-old stage,CCVy?k_Hpi
001C_AAACGGGAGGCTCATT-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,NaN,macrophage,10x 3' v2,normal,Homo sapiens,male,lung,unknown,22-year-old stage,dICnrm?v}Y
001C_AAACGGGAGGGAACGG-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Monocyte classical,Monocyte,singlet,...,NaN,classical monocyte,10x 3' v2,normal,Homo sapiens,male,lung,unknown,22-year-old stage,5Vz7FxxH^r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACATCTATGG-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,NaN,macrophage,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,64-year-old stage,A#pCiVqT8Z
TTTGTCACATGTTGAC-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Monocyte classical,NaN,singlet,...,NaN,classical monocyte,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,64-year-old stage,C{<h|YtC)P
TTTGTCAGTGTTGGGA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,NaN,macrophage,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,64-year-old stage,lpE0#RH%^*
TTTGTCATCAGTTTGG-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,T cell regulatory,NaN,singlet,...,NaN,regulatory T cell,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,64-year-old stage,)854XipjN$


In [6]:
adata.obs.columns

Index(['sample', 'uicc_stage', 'ever_smoker', 'age', 'donor_id', 'origin',
       'dataset', 'ann_fine', 'cell_type_predicted', 'doublet_status',
       'leiden', 'n_genes_by_counts', 'total_counts', 'total_counts_mito',
       'pct_counts_mito', 'ann_coarse', 'cell_type_tumor', 'tumor_stage',
       'TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation',
       'KRAS_mutation', 'ROS_mutation', 'origin_fine', 'study', 'platform',
       'cell_type_major', 'cell_type_neutro', 'cell_type_neutro_coarse',
       'suspension_type', 'assay_ontology_term_id',
       'cell_type_ontology_term_id', 'development_stage_ontology_term_id',
       'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id',
       'tissue_ontology_term_id', 'tissue_type', 'EGFR_mutation', 'cell_type',
       'assay', 'disease', 'organism', 'sex', 'tissue',
       'self_reported_ethnicity', 'development_stage', 'obser

In [23]:
list(adata.obs['cell_type_tumor'].unique())

['Monocyte non-classical',
 'Macrophage alveolar',
 'Endothelial cell lymphatic',
 'Macrophage',
 'Monocyte classical',
 'Tumor cells LUAD',
 'T cell CD8 dividing',
 'myeloid dividing',
 'NK cell',
 'cDC2',
 'B cell',
 'Endothelial cell capillary',
 'T cell regulatory',
 'ROS1+ healthy epithelial',
 'Alveolar cell type 1',
 'Endothelial cell venous',
 'T cell CD4',
 'Endothelial cell arterial',
 'Fibroblast adventitial',
 'Plasma cell',
 'T cell CD8 naive',
 'Smooth muscle cell',
 'Ciliated',
 'transitional club/AT2',
 'T cell CD8 effector memory',
 'Club',
 'T cell CD4 dividing',
 'T cell NK-like',
 'Mesothelial',
 'Fibroblast alveolar',
 'Alveolar cell type 2',
 'T cell CD8 activated',
 'Tumor cells LUSC mitotic',
 'NK cell dividing',
 'cDC1',
 'Mast cell',
 'stromal dividing',
 'Plasma cell dividing',
 'pDC',
 'T cell CD8 terminally exhausted',
 'DC mature',
 'Pericyte',
 'Fibroblast peribronchial',
 'Tumor cells LUAD mitotic',
 'Neutrophils',
 'Tumor cells LUAD EMT',
 'Tumor cells 

# one set of simulations for lung cancer cells
- we take each lung cancer cell by mutation, then simulate growth artrest
- we need top account for tumour celltypes, mutation status, and tumour stage in a way that makes sense
- we then do UMAP and KDE (scany compute embedding density) 
- Plot whther a simulation splits the simulated data from paired ground truth in embedding space (e.g., on scGPT/GF embeddings, make this dynamic so we can use any embedding)
- Compute metrics to show splits for each pathway simulation and celltye
- We show split of simulation vs baseline and groun truth on UMAP, (scanpy embedding densitry)
- We show DEGs dotplot from scanpy (sc.tl.rank_genes) -> (sc.tl.dotplot) then compute the DEGs agreement score for top 100DEGs

- basically also want to show which of these celltypes under which mutation status changes significantly in UMAP space as well. 

- We might only select a subset to display at the end so account for this and enable us to assess all angles effectively. 

In [24]:
# Let's focus on lung adenocarcinoma & only select cancer cells
meta = adata.obs.copy()
meta = meta[(meta['disease'].isin((['lung adenocarcinoma']))) & (meta['cell_type_tumor'].isin((['Tumor cells LUSC mitotic','Tumor cells LUAD mitotic',
 'Tumor cells LUAD EMT',
 'Tumor cells NSCLC mixed',
 'Tumor cells LUSC',
 'Tumor cells LUAD NE',
 'Tumor cells LUAD MSLN'])))]

# accouint for mutation state, we should do perturbations per mutation state
mut_cols = ['TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation',
       'KRAS_mutation', 'ROS_mutation','EGFR_mutation']

tum_stage = 'tumor_stage'



# Example gene shift logic
hallmark_up = [
    "HALLMARK_P53_PATHWAY", "HALLMARK_APOPTOSIS", "HALLMARK_UNFOLDED_PROTEIN_RESPONSE",
    "HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY", "HALLMARK_INTERFERON_GAMMA_RESPONSE"
]
hallmark_down = [
    "HALLMARK_G2M_CHECKPOINT", "HALLMARK_E2F_TARGETS", "HALLMARK_MYC_TARGETS_V1",
    "HALLMARK_MTORC1_SIGNALING"
]

In [ ]:
# write the data for embeddings
adata[adata.obs.index.isin(meta.index)].write('./data/sim_data.h5ad')

## Get embedding

In [18]:
#############################################
# 1) initialise config
#############################################
with initialize(version_base=None, config_path="./config"):
    config = compose(config_name="inference.yaml")
    exp_details = config['experiment_details']
    config_scgpt = InferenceConfig(**config['scgpt'])
    config_gf_i4096 = InferenceConfig(**config['gf-12L-95M-i4096'])
# Configs can be set either through the .yaml file, or directly in-line as below
for key in config:
    print("Configuration for : {}".format(key))
    print(config[key])

Configuration for : experiment_details
{'experiment_name': 'pbmc_scGPT_GF_inference', 'experiment_description': 'An experiment constructed using data from ALS patients from the B4 motor cortex for the purposes of evaluating cellstate recovery for each model'}
Configuration for : scgpt
{'data': '../data/A1_V1_PBMC.h5ad', 'model_name': 'scgpt', 'model_path': None, 'output_dir': '../outputs/inference_output', 'prediction_type': 'get_embeddings', 'batch_size': 24, 'embed_mode': 'cell', 'species': 'human', 'device': 'cuda'}
Configuration for : gf-12L-30M-i2048
{'data': '../data/A1_V1_PBMC.h5ad', 'model_name': 'gf-12L-30M-i2048', 'model_path': None, 'output_dir': '../outputs/inference_output', 'prediction_type': 'get_embeddings', 'batch_size': 24, 'embed_mode': 'cell', 'species': 'human', 'device': 'cuda'}
Configuration for : gf-12L-95M-i4096
{'data': '../data/A1_V1_PBMC.h5ad', 'model_name': 'gf-12L-95M-i4096-CLcancer', 'model_path': None, 'output_dir': '../outputs/inference_output', 'predic

In [ ]:
# Get embeddings
# scGPT
config_scgpt.data = './data/sim_data.h5ad'
result_scGPT = run_inference(config_scgpt)

# config_gf_i4096
config_gf_i4096.data = './data/sim_data.h5ad'
result_gf_i4096 = run_inference(config_gf_i4096)


In [35]:
adata.var

,is_highly_variable,mito,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length,feature_type
ENSG00000121410,True,False,86866,2.527138,90.681054,2355657.0,False,A1BG_ENSG00000121410,NCBITaxon:9606,gene,2134,protein_coding
ENSG00000268895,True,False,12257,0.369194,98.685074,344142.0,False,A1BG-AS1,NCBITaxon:9606,gene,1667,lncRNA
ENSG00000175899,True,False,122241,22.579874,86.886039,21047694.0,False,A2M_ENSG00000175899,NCBITaxon:9606,gene,590,protein_coding
ENSG00000245105,False,False,8827,0.172223,99.053043,160537.0,False,A2M-AS1,NCBITaxon:9606,gene,2551,lncRNA
ENSG00000166535,True,False,5096,0.038011,99.453303,35432.0,False,A2ML1_ENSG00000166535,NCBITaxon:9606,gene,592,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000070476,False,False,43291,0.442399,95.355760,412380.0,False,ZXDC_ENSG00000070476,NCBITaxon:9606,gene,3006,protein_coding
ENSG00000203995,False,False,3517,0.009742,99.622698,9081.0,False,ZYG11A_ENSG00000203995,NCBITaxon:9606,gene,4193,protein_coding
ENSG00000162378,False,False,71512,0.275276,92.328224,256597.0,False,ZYG11B_ENSG00000162378,NCBITaxon:9606,gene,5218,protein_coding
ENSG00000159840,False,False,205370,4.536451,77.967996,4228626.0,False,ZYX_ENSG00000159840,NCBITaxon:9606,gene,814,protein_coding


## generate UMAP and metrics

# One set of simulations for Tcells
- We take ground truth CD8 tcells ('T cell CD8 effector memory') and exhausted CD8 tcells ('T cell CD8 terminally exhausted')
- We use hallmark to determine which genes to simulate pertubations for
- We compute embeddings (scGPT, etc)
- We show split of simulation vs baseline and groun truth on UMAP, (scanpy embedding densitry)
- We show DEGs dotplot from scanpy (sc.tl.rank_genes) -> (sc.tl.dotplot) then compute the DEGs agreement score for top 100DEGs

In [ ]:
# Let's focus on lung adenocarcinoma & only select cancer cells
meta = adata.obs.copy()
meta = meta[(meta['disease'].isin((['lung adenocarcinoma']))) & (meta['ann_fine'].isin((['T cell CD8 effector memory','T cell CD8 terminally exhausted'])))]

# accouint for mutation state, we should do perturbations per mutation state
mut_cols = ['TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation',
       'KRAS_mutation', 'ROS_mutation','EGFR_mutation']

# xxxx